In [1]:
import pandas as pd
import numpy as np
import time
import os
import zipfile
import pandas_gbq
import glob
from google.cloud import bigquery
import re

In [2]:
client = bigquery.Client()

In [3]:
query = """ 
        SELECT ANO as ano, MES as mes, TIPO_APOSENTADORIA as tipo_aposentadoria, 
            CASE WHEN ORGSUP_LOTACAO ="Sem informação" THEN ORG_LOTACAO ELSE ORGSUP_LOTACAO 
            END as org_lotacao, 'REGIME JURIDICO UNICO' as regime_juridico,
            1 as total FROM `repositoriodedadosgpsp.portal_transparencia_cgu.2020_nov_siape_aposentados_cadastro` 
            UNION ALL
        SELECT ANO as ano, MES as mes, TIPO_APOSENTADORIA as tipo_aposentadoria, 
            CASE WHEN ORGSUP_LOTACAO ="Sem informação" THEN ORG_LOTACAO ELSE ORGSUP_LOTACAO 
            END as org_lotacao, 'REGIME JURIDICO UNICO' as regime_juridico,
            1 as total FROM `repositoriodedadosgpsp.portal_transparencia_cgu.2021_nov_siape_aposentados_cadastro` 
            UNION ALL
        SELECT ANO as ano, MES as mes, TIPO_APOSENTADORIA as tipo_aposentadoria, 
            CASE WHEN ORGSUP_LOTACAO ="Sem informação" THEN ORG_LOTACAO ELSE ORGSUP_LOTACAO 
            END as org_lotacao, 'REGIME JURIDICO UNICO' as regime_juridico,
            1 as total FROM `repositoriodedadosgpsp.portal_transparencia_cgu.2022_nov_siape_aposentados_cadastro`
            UNION ALL
        SELECT ANO as ano, MES as mes, TIPO_APOSENTADORIA as tipo_aposentadoria, 
            CASE WHEN ORGSUP_LOTACAO ="Sem informação" THEN ORG_LOTACAO ELSE ORGSUP_LOTACAO 
            END as org_lotacao, 'REGIME JURIDICO UNICO' as regime_juridico,
            1 as total FROM `repositoriodedadosgpsp.portal_transparencia_cgu.2023_nov_siape_aposentados_cadastro`
        """  

In [4]:
df = pandas_gbq.read_gbq(query, project_id='repositoriodedadosgpsp')
df

Downloading: 100%|██████████|


,ano,mes,tipo_aposentadoria,org_lotacao,regime_juridico,total
0,2022,11,APOSENTADORIA POR INVALIDEZ,Ministério da Economia,REGIME JURIDICO UNICO,1
1,2022,11,APOSENTADORIA VOLUNTARIA,Ministério da Economia,REGIME JURIDICO UNICO,1
2,2022,11,APOSENTADORIA VOLUNTARIA,Ministério da Economia,REGIME JURIDICO UNICO,1
3,2022,11,APOSENTADORIA VOLUNTARIA,Ministério da Economia,REGIME JURIDICO UNICO,1
4,2022,11,APOSENTADORIA VOLUNTARIA,Ministério da Economia,REGIME JURIDICO UNICO,1
...,...,...,...,...,...,...
1689358,2021,11,APOSENTADORIA VOLUNTARIA,Ministério da Economia,REGIME JURIDICO UNICO,1
1689359,2021,11,APOSENTADORIA VOLUNTARIA,Ministério da Economia,REGIME JURIDICO UNICO,1
1689360,2021,11,APOSENTADORIA POR INVALIDEZ,Ministério da Defesa,REGIME JURIDICO UNICO,1
1689361,2021,11,APOSENTADORIA POR INVALIDEZ,Ministério da Defesa,REGIME JURIDICO UNICO,1


In [5]:
df.columns

Index(['ano', 'mes', 'tipo_aposentadoria', 'org_lotacao', 'regime_juridico',
       'total'],
      dtype='object')

In [6]:
df['tipo_aposentadoria'].unique()

array(['APOSENTADORIA POR INVALIDEZ', 'APOSENTADORIA VOLUNTARIA',
       'APOSENTADORIA COMPULSORIA', 'APOSENTADORIA POR INCAPACIDADE',
       'OUTROS'], dtype=object)

In [7]:
def renomear(var_aposentadoria):
    var_aposentadoria = re.sub(r"\bAPOSENTADORIA POR INVALIDEZ\b", "Aposentadoria por invalidez", var_aposentadoria)
    var_aposentadoria = re.sub(r"\bAPOSENTADORIA VOLUNTARIA\b", "Aposentadoria voluntária", var_aposentadoria)
    var_aposentadoria = re.sub(r"\bAPOSENTADORIA COMPULSORIA\b", "Aposentadoria compulsória", var_aposentadoria)
    var_aposentadoria = re.sub(r"\bAPOSENTADORIA POR INCAPACIDADE\b", "Aposentadoria por incapacidade", var_aposentadoria)
    var_aposentadoria = re.sub(r"\bOUTROS\b", "Outros", var_aposentadoria)
    
    return var_aposentadoria

In [8]:
df['tipo_aposentadoria'] = df['tipo_aposentadoria'].apply(renomear)

In [9]:
df['tipo_aposentadoria'].unique()

array(['Aposentadoria por invalidez', 'Aposentadoria voluntária',
       'Aposentadoria compulsória', 'Aposentadoria por incapacidade',
       'Outros'], dtype=object)

In [10]:
df['org_lotacao'] = df['org_lotacao'].str.title()

In [11]:
df['org_lotacao'].unique()

array(['Ministério Da Economia', 'Ministério Da Saúde',
       'Ministério Da Educação',
       'Ministério Da Justiça E Segurança Pública',
       'Ministério Da Defesa',
       'Ministério Da Ciência, Tecnologia, Inovações E Comunicações',
       'Ministério Da Infraestrutura', 'Presidência Da República',
       'Ministério Das Relações Exteriores',
       'Ministério Da Agricultura, Pecuária E Abastecimento',
       'Ministério Do Desenvolvimento Regional', 'Ministério Do Turismo',
       'Ministério Do Meio Ambiente', 'Ministério De Minas E Energia',
       'Controladoria-Geral Da União',
       'Ministério Do Desenvolvimento Social',
       'Ministerio Das Comunicacoes', 'Defensoria Pública Da União',
       'Ministerio Da Cultura', 'Min Gestao E Inov Em Serv Publicos',
       'Ministerio Dos Transportes', 'Ministerio Da Previdencia Social',
       'Ministerio Do Planejamento E Orcamento',
       'Min Do Desenv Agr E Agric Familiar',
       'Min Da Integ E Do Desenv Regional',
   

In [12]:
def renomear2(var_aposentadoria):
    var_aposentadoria = re.sub(r"\bMin Gestao E Inov Em Serv Publicos\b", "Ministério Gestão E Inovação Em Serviços Públicos", var_aposentadoria)
    var_aposentadoria = re.sub(r"\bMinisterio Dos Povos Indigenas\b", "Ministério dos Povos Indígenas", var_aposentadoria)
    var_aposentadoria = re.sub(r"\bMinisterio Da Previdencia Social\b", "Ministério Da Previdência Social", var_aposentadoria)
    var_aposentadoria = re.sub(r"\bMinisterio Do Planejamento E Orcamento\b", "Ministério Do Planejamento E Orçamento", var_aposentadoria)
    var_aposentadoria = re.sub(r"\bMin Da Integ E Do Desenv Regional\b", "Ministério da Integração e do Desenvolvimento Regional", var_aposentadoria)
    var_aposentadoria = re.sub(r"\bMin Do Desenv Agr E Agric Familiar\b", "Ministério do Desenvolvimento Agrário e Agricultura Familiar", var_aposentadoria)
    var_aposentadoria = re.sub(r"\bMin Desenvolv Ind Comercio E Servicos\b", "Ministério do Desenvolvimento, Indústria, Comércio e Serviços", var_aposentadoria)
    var_aposentadoria = re.sub(r"\bComunicacoes\b", "Comunicações", var_aposentadoria)
    var_aposentadoria = re.sub(r"\bMinisterio\b", "Ministério", var_aposentadoria)
    var_aposentadoria = re.sub(r"\bPrevidencia\b", "Previdência", var_aposentadoria)

    
    return var_aposentadoria

In [13]:
df['org_lotacao'] = df['org_lotacao'].apply(renomear2)

In [14]:
df['org_lotacao'].unique()

array(['Ministério Da Economia', 'Ministério Da Saúde',
       'Ministério Da Educação',
       'Ministério Da Justiça E Segurança Pública',
       'Ministério Da Defesa',
       'Ministério Da Ciência, Tecnologia, Inovações E Comunicações',
       'Ministério Da Infraestrutura', 'Presidência Da República',
       'Ministério Das Relações Exteriores',
       'Ministério Da Agricultura, Pecuária E Abastecimento',
       'Ministério Do Desenvolvimento Regional', 'Ministério Do Turismo',
       'Ministério Do Meio Ambiente', 'Ministério De Minas E Energia',
       'Controladoria-Geral Da União',
       'Ministério Do Desenvolvimento Social',
       'Ministério Das Comunicações', 'Defensoria Pública Da União',
       'Ministério Da Cultura',
       'Ministério Gestão E Inovação Em Serviços Públicos',
       'Ministério Dos Transportes', 'Ministério Da Previdência Social',
       'Ministério Do Planejamento E Orçamento',
       'Ministério do Desenvolvimento Agrário e Agricultura Familiar',

In [15]:
df['regime_juridico'].unique()

array(['REGIME JURIDICO UNICO'], dtype=object)

In [16]:
def renomear3(var_regime):
    var_regime = re.sub(r"\bREGIME JURIDICO UNICO\b", "Regime Jurídico Único", var_regime)
    return var_regime

In [17]:
df['regime_juridico'] = df['regime_juridico'].apply(renomear3)

In [18]:
df['org_lotacao'].unique()

array(['Ministério Da Economia', 'Ministério Da Saúde',
       'Ministério Da Educação',
       'Ministério Da Justiça E Segurança Pública',
       'Ministério Da Defesa',
       'Ministério Da Ciência, Tecnologia, Inovações E Comunicações',
       'Ministério Da Infraestrutura', 'Presidência Da República',
       'Ministério Das Relações Exteriores',
       'Ministério Da Agricultura, Pecuária E Abastecimento',
       'Ministério Do Desenvolvimento Regional', 'Ministério Do Turismo',
       'Ministério Do Meio Ambiente', 'Ministério De Minas E Energia',
       'Controladoria-Geral Da União',
       'Ministério Do Desenvolvimento Social',
       'Ministério Das Comunicações', 'Defensoria Pública Da União',
       'Ministério Da Cultura',
       'Ministério Gestão E Inovação Em Serviços Públicos',
       'Ministério Dos Transportes', 'Ministério Da Previdência Social',
       'Ministério Do Planejamento E Orçamento',
       'Ministério do Desenvolvimento Agrário e Agricultura Familiar',

In [19]:
ministerios = {
    "Ministério Do Turismo": "Cultura, Turismo e Esportes",
    "Ministério Da Economia": "Economia, Finanças, Planejamento, Previdência e Gestão",
    "Ministério Da Saúde": "Saúde",
    "Ministério Da Justiça E Segurança Pública": "Justiça, Segurança e Órgãos de Controle e Defesa",
    "Ministério Da Educação": "Educação e Ciência e Tecnologia",
    "Ministério De Minas E Energia": "Infraestrutura, Transporte, Desenvolvimento Regional, Portos e Aeroportos, Minas e Energia",
    "Ministério Da Defesa": "Justiça, Segurança e Órgãos de Controle e Defesa",
    "Ministério Da Ciência, Tecnologia, Inovações E Comunicações": "Educação e Ciência e Tecnologia",
    "Ministério Da Infraestrutura": "Infraestrutura, Transporte, Desenvolvimento Regional, Portos e Aeroportos, Minas e Energia",
    "Presidência Da República": "Presidência Da República",
    "Ministério Das Relações Exteriores": "Relações Exteriores",
    "Ministério Da Agricultura, Pecuária E Abastecimento": "Meio ambiente, Agricultura e Agropecuária",
    "Ministério Do Desenvolvimento Regional": "Infraestrutura, Transporte, Desenvolvimento Regional, Portos e Aeroportos, Minas e Energia",
    "Ministério Do Meio Ambiente": "Meio ambiente, Agricultura e Agropecuária",
    "Controladoria-Geral Da União": "Justiça, Segurança e Órgãos de Controle e Defesa",
    "Ministério Do Desenvolvimento Social": "Desenvolvimento Social e Direitos Humanos",
    "Defensoria Pública Da União": "Justiça, Segurança e Órgãos de Controle e Defesa",
    "Conselho Nacional De Desenvolvimento Científico E Tecnológico": "Educação e Ciência e Tecnologia",
    "Ministério Gestão E Inovação Em Serviços Públicos": "Economia, Finanças, Planejamento, Previdência e Gestão",
    "Ministério Da Cultura": "Cultura, Turismo e Esportes",
    "Ministério Dos Transportes": "Infraestrutura, Transporte, Desenvolvimento Regional, Portos e Aeroportos, Minas e Energia",
    "Ministério Da Previdência Social": "Economia, Finanças, Planejamento, Previdência e Gestão",
    "Ministério Do Planejamento E Orçamento": "Economia, Finanças, Planejamento, Previdência e Gestão",
    "Ministério da Integração e do Desenvolvimento Regional": "Infraestrutura, Transporte, Desenvolvimento Regional, Portos e Aeroportos, Minas e Energia",
    "Ministério do Desenvolvimento Agrário e Agricultura Familiar": "Meio ambiente, Agricultura e Agropecuária",
    "Ministério dos Povos Indígenas": "Desenvolvimento Social e Direitos Humanos",
    "Ministério Do Trabalho E Emprego": "Economia, Finanças, Planejamento, Previdência e Gestão",
    "Ministério De Portos E Aeroportos": "Infraestrutura, Transporte, Desenvolvimento Regional, Portos e Aeroportos, Minas e Energia",
    "Ministério do Desenvolvimento, Indústria, Comércio e Serviços": "Infraestrutura, Transporte, Desenvolvimento Regional, Portos e Aeroportos, Minas e Energia",
    "Ministério Da Fazenda": "Economia, Finanças, Planejamento, Previdência e Gestão",
    "Ministério Das Comunicações": "Infraestrutura, Transporte, Desenvolvimento Regional, Portos e Aeroportos, Minas e Energia",
    "Caixa De Financiamento Imobiliário Da Aeronáutica": "Economia, Finanças, Planejamento, Previdência e Gestão",
    "Ministério Do Trabalho E Previdência": "Economia, Finanças, Planejamento, Previdência e Gestão"
}


In [20]:
def categorizando(x):
    if x in ministerios:
        return ministerios[x]


In [21]:
df['categoria_ministerios'] = df['org_lotacao'].apply(categorizando)

In [22]:
df

,ano,mes,tipo_aposentadoria,org_lotacao,regime_juridico,total,categoria_ministerios
0,2022,11,Aposentadoria por invalidez,Ministério Da Economia,Regime Jurídico Único,1,"Economia, Finanças, Planejamento, Previdência ..."
1,2022,11,Aposentadoria voluntária,Ministério Da Economia,Regime Jurídico Único,1,"Economia, Finanças, Planejamento, Previdência ..."
2,2022,11,Aposentadoria voluntária,Ministério Da Economia,Regime Jurídico Único,1,"Economia, Finanças, Planejamento, Previdência ..."
3,2022,11,Aposentadoria voluntária,Ministério Da Economia,Regime Jurídico Único,1,"Economia, Finanças, Planejamento, Previdência ..."
4,2022,11,Aposentadoria voluntária,Ministério Da Economia,Regime Jurídico Único,1,"Economia, Finanças, Planejamento, Previdência ..."
...,...,...,...,...,...,...,...
1689358,2021,11,Aposentadoria voluntária,Ministério Da Economia,Regime Jurídico Único,1,"Economia, Finanças, Planejamento, Previdência ..."
1689359,2021,11,Aposentadoria voluntária,Ministério Da Economia,Regime Jurídico Único,1,"Economia, Finanças, Planejamento, Previdência ..."
1689360,2021,11,Aposentadoria por invalidez,Ministério Da Defesa,Regime Jurídico Único,1,"Justiça, Segurança e Órgãos de Controle e Defesa"
1689361,2021,11,Aposentadoria por invalidez,Ministério Da Defesa,Regime Jurídico Único,1,"Justiça, Segurança e Órgãos de Controle e Defesa"


In [23]:
df['categoria_ministerios'].unique()

array(['Economia, Finanças, Planejamento, Previdência e Gestão', 'Saúde',
       'Educação e Ciência e Tecnologia',
       'Justiça, Segurança e Órgãos de Controle e Defesa',
       'Infraestrutura, Transporte, Desenvolvimento Regional, Portos e Aeroportos, Minas e Energia',
       'Presidência Da República', 'Relações Exteriores',
       'Meio ambiente, Agricultura e Agropecuária',
       'Cultura, Turismo e Esportes',
       'Desenvolvimento Social e Direitos Humanos'], dtype=object)

In [24]:
df['categoria_ministerios'].value_counts()

Educação e Ciência e Tecnologia                                                               500246
Economia, Finanças, Planejamento, Previdência e Gestão                                        472829
Saúde                                                                                         397423
Justiça, Segurança e Órgãos de Controle e Defesa                                              174720
Meio ambiente, Agricultura e Agropecuária                                                      69864
Presidência Da República                                                                       30159
Infraestrutura, Transporte, Desenvolvimento Regional, Portos e Aeroportos, Minas e Energia     26970
Cultura, Turismo e Esportes                                                                     8578
Relações Exteriores                                                                             6644
Desenvolvimento Social e Direitos Humanos                                                  

In [25]:
df['org_lotacao'].unique()


array(['Ministério Da Economia', 'Ministério Da Saúde',
       'Ministério Da Educação',
       'Ministério Da Justiça E Segurança Pública',
       'Ministério Da Defesa',
       'Ministério Da Ciência, Tecnologia, Inovações E Comunicações',
       'Ministério Da Infraestrutura', 'Presidência Da República',
       'Ministério Das Relações Exteriores',
       'Ministério Da Agricultura, Pecuária E Abastecimento',
       'Ministério Do Desenvolvimento Regional', 'Ministério Do Turismo',
       'Ministério Do Meio Ambiente', 'Ministério De Minas E Energia',
       'Controladoria-Geral Da União',
       'Ministério Do Desenvolvimento Social',
       'Ministério Das Comunicações', 'Defensoria Pública Da União',
       'Ministério Da Cultura',
       'Ministério Gestão E Inovação Em Serviços Públicos',
       'Ministério Dos Transportes', 'Ministério Da Previdência Social',
       'Ministério Do Planejamento E Orçamento',
       'Ministério do Desenvolvimento Agrário e Agricultura Familiar',

In [26]:
df['categoria_ministerios'].unique()

array(['Economia, Finanças, Planejamento, Previdência e Gestão', 'Saúde',
       'Educação e Ciência e Tecnologia',
       'Justiça, Segurança e Órgãos de Controle e Defesa',
       'Infraestrutura, Transporte, Desenvolvimento Regional, Portos e Aeroportos, Minas e Energia',
       'Presidência Da República', 'Relações Exteriores',
       'Meio ambiente, Agricultura e Agropecuária',
       'Cultura, Turismo e Esportes',
       'Desenvolvimento Social e Direitos Humanos'], dtype=object)

In [62]:
df.columns

Index(['ano', 'mes', 'tipo_aposentadoria', 'org_lotacao', 'regime_juridico',
       'total', 'categoria_ministerios'],
      dtype='object')

In [61]:
client = bigquery.Client()
dataset_ref = client.dataset('perfil_remuneracao')

In [63]:
#schema=[bigquery.SchemaField('ano','INTEGER',description='Ano de referência da observação'),
        bigquery.SchemaField('mes','INTEGER',description='Mês de referência da observação'),        
        bigquery.SchemaField('tipo_aposentadoria','STRING',description='Tipo de aposentadoria'),
        bigquery.SchemaField('org_lotacao','STRING',description='Órgão de lotação'),
        bigquery.SchemaField('regime_juridico','STRING',description='Regime jurídico'),
        bigquery.SchemaField('total','INTEGER',description='Quantidade total da observação'),
        bigquery.SchemaField('categoria_ministerios','STRING',description='Categorias dos ministérios')]


In [64]:
table_ref = dataset_ref.table('SIAPE_aposentadorias') 
job_config = bigquery.LoadJobConfig(schema=schema)
job = client.load_table_from_dataframe(df, table_ref, job_config=job_config)
job.result() 


LoadJob<project=repositoriodedadosgpsp, location=US, id=49644a91-c758-48c5-a630-bc40fc8eb799>